In [6]:
%load_ext autoreload
%autoreload 2
     
import torch
import os
from csng.brainreader_mouse.data import get_brainreader_mouse_dataloaders


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:

device = os.environ["DEVICE"]
DATA_DIR = os.environ['DATA_PATH']
CACHE_DIR = os.path.join(DATA_DIR, 'brainreader_cache')

utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils', )

Using cache found in /home/vanousek/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [8]:
### brainreader mouse data
config["data"]["brainreader_mouse"] = {
    "device": config["device"],
    "mixing_strategy": config["data"]["mixing_strategy"],
    "max_batches": None,
    "data_dir": os.path.join(DATA_PATH_BRAINREADER, "data"),
    "batch_size": 128,
    "sessions": list(range(1, 23)),
    # "sessions": [6],
    "resize_stim_to": (36, 64),
    "normalize_stim": True,
    "normalize_resp": False,
    "div_resp_by_std": True,
    "clamp_neg_resp": False,
    "additional_keys": None,
    
    "avg_test_resp": True,
    


SyntaxError: incomplete input (2238914678.py, line 17)